In [ ]:
import pandas as pd
import sqlite3
import gdown
import os.path
import csv

In [ ]:
csv_file = 'tennis_data.csv'
if not os.path.isfile(csv_file):
    gdown.download(id='1k-aEbSDXA7SvyPoHD6AjFgfo79dfmkAI', output=csv_file)

In [ ]:
#df = pd.read_csv(csv_file)

First technique, all in a single table and from there we insert into other tables.

In [ ]:
with open(csv_file) as f:
    rdr = csv.reader(f)
    headers = next(rdr)
    rows = []
    for row in rdr:
        rows.append(row)

In [ ]:
connection = sqlite3.connect('tennis_data.db')
cursor = connection.cursor()

In [ ]:
query = """
INSERT INTO report (ATP, Location, Tournament, Date, Series, Court, Surface, Round, "Best of", Winner, Loser, WRank, LRank, WPts, LPts, W1, L1, W2, L2, W3, L3, W4, L4, W5, L5, Comment, pl1_flag, pl1_year_pro, pl1_weight, pl1_height, pl1_hand, pl2_flag, pl2_year_pro, pl2_weight, pl2_height, pl2_hand)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""
cursor.executemany(query, rows)
connection.commit()

## Glitches

The CSV file contains some glitches that complicate the relationships between the chosen table structure. Before starting to feed the data to the tables lets cleanup the data first
